# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../WeatherPy/output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,constantia,44.1833,28.6500,20.38,62,1,2.67,RO,1691037591
1,1,tiksi,71.6872,128.8694,5.54,89,100,6.28,RU,1691037721
2,2,edinburgh of the seven seas,-37.0676,-12.3116,13.91,83,96,17.53,SH,1691037721
3,3,yellowknife,62.4560,-114.3525,18.29,59,75,3.09,CA,1691037661
4,4,port lincoln,-34.7333,135.8667,22.04,43,80,10.40,AU,1691037722


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [8]:
#%%capture --no-display

# Configure the map
humidity_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = .5,
    color = "City",
)

# Display the map plot
humidity_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = (city_data_df.loc[(city_data_df["Max Temp"]<30) & (city_data_df["Max Temp"]>22) &
                                (city_data_df["Humidity"]<85) & (city_data_df["Humidity"]>55) &
                                (city_data_df["Cloudiness"]<12)])

# Drop any rows with null values
city_data_df = city_data_df.dropna()

# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,anna regina,7.2644,-58.5077,26.09,82,3,1.47,GY,1691037723
16,16,san patricio,28.0170,-97.5169,29.92,75,11,5.17,US,1691037724
29,29,belyy yar,53.6039,91.3903,26.94,57,0,1.00,RU,1691037727
35,35,ypsonas,34.6884,32.9574,26.54,74,0,1.03,CY,1691037728
63,63,an nasr,31.2814,34.3025,26.75,65,0,1.74,PS,1691037734
70,70,la'ie,21.6477,-157.9253,28.98,71,3,7.18,US,1691037735
84,84,uturoa,-16.7333,-151.4333,26.28,66,8,5.67,PF,1691037269
90,90,wailua homesteads,22.0669,-159.3780,29.24,78,0,6.69,US,1691037270
94,94,puerto baquerizo moreno,-0.9000,-89.6000,23.82,79,6,4.65,EC,1691037568
101,101,sirte,31.2089,16.5887,25.78,76,0,1.13,LY,1691037742


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df.copy()
hotel_df = hotel_df[["City", "Country", "Lat", "Lng", "Humidity"]]

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
9,anna regina,GY,7.2644,-58.5077,82,
16,san patricio,US,28.0170,-97.5169,75,
29,belyy yar,RU,53.6039,91.3903,57,
35,ypsonas,CY,34.6884,32.9574,74,
63,an nasr,PS,31.2814,34.3025,65,
70,la'ie,US,21.6477,-157.9253,71,
84,uturoa,PF,-16.7333,-151.4333,66,
90,wailua homesteads,US,22.0669,-159.3780,78,
94,puerto baquerizo moreno,EC,-0.9000,-89.6000,79,
101,sirte,LY,31.2089,16.5887,76,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [6]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":1
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, "Lat"]
    longitude = hotel_df.loc[index, "Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head(20)

Starting hotel search
anna regina - nearest hotel: Jaigobin Hotel
san patricio - nearest hotel: No hotel found
belyy yar - nearest hotel: No hotel found
ypsonas - nearest hotel: Club Aphrodite
an nasr - nearest hotel: صالة قصر اللوتس
la'ie - nearest hotel: No hotel found
uturoa - nearest hotel: Hawaiki Nui hotel
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
puerto baquerizo moreno - nearest hotel: Dory's House
sirte - nearest hotel: فندق المدينة-سرت
papao - nearest hotel: Hiti Moana Villa Lodge
port mathurin - nearest hotel: Escale Vacances
lihue - nearest hotel: Kauai Palms
kapa'a - nearest hotel: Pono Kai Resort
dzilam gonzalez - nearest hotel: No hotel found
bozyazi - nearest hotel: Bozyazı Öğretmenevi
mejit - nearest hotel: No hotel found
laguna - nearest hotel: Holiday Inn Express & Suites
caibarien - nearest hotel: Restaurante Hostal El Coliseo
newman - nearest hotel: No hotel found
roswell - nearest hotel: Best Western Plus Hotel
misratah - nearest hotel:

,City,Country,Lat,Lng,Humidity,Hotel Name
9,anna regina,GY,7.2644,-58.5077,82,Jaigobin Hotel
16,san patricio,US,28.0170,-97.5169,75,No hotel found
29,belyy yar,RU,53.6039,91.3903,57,No hotel found
35,ypsonas,CY,34.6884,32.9574,74,Club Aphrodite
63,an nasr,PS,31.2814,34.3025,65,صالة قصر اللوتس
70,la'ie,US,21.6477,-157.9253,71,No hotel found
84,uturoa,PF,-16.7333,-151.4333,66,Hawaiki Nui hotel
90,wailua homesteads,US,22.0669,-159.3780,78,Hilton Garden Inn Kauai Wailua Bay
94,puerto baquerizo moreno,EC,-0.9000,-89.6000,79,Dory's House
101,sirte,LY,31.2089,16.5887,76,فندق المدينة-سرت


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [9]:
#%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    alpha = .5,
    color = "City",
)

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)